In [10]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

# Scouting Analysis

This notebook offers a scouting analysis tool for new player acquisitions using financial data from Transfermarkt and player statistics from Wyscout.

- Financial data from Transfermarkt: Transfer fees, market value
- Player statistics from Wyscout: Player position, age, minutes played, goals, assists, shots, passes, tackles, interceptions, etc.

# 1. Data Collection

## 1.1. Transfermarkt

In [23]:
def to_number(string):
    """Converts a string to a number, e.g. '€1.2m' to 1.2"""
    try:
        if string[-1] == 'm':
            # convert to millions
            return float(string[1:-1])
        else:
            return float(string[1:-1]) / 1000
    except:
        return np.nan


# Get out Player Name, Position, Left Club, Joined Club, Fee
player_names = []
player_fees = []
player_values = []

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

base_url = "https://www.transfermarkt.co.uk/transfers/transferrekorde/statistik?saison_id=2017&land_id=0&ausrichtung=&spielerposition_id=&altersklasse=&leihe=&w_s=&plus=1&page="

# Loop through multiple pages
page_number = 1
end = False
limit = 5

while end == False:
    page = base_url + str(page_number)
    try:
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
        player_rows = pageSoup.find_all("tr", {"class": ["odd", "even"]})
        page_number += 1

        for player_row in player_rows:
            player_name = player_row.select("a")[0].text
            player_fee = to_number(player_row.select("td")[17].text)
            player_value = to_number(player_row.select("td")[6].text)

            player_names.append(player_name)
            player_fees.append(player_fee)
            player_values.append(player_value)
    
        if limit is not None:
            if page_number == limit:
                end = True
                print("End of pages", page_number)

    except Exception as e:
        print(e)
        end = True
        print("End of pages", page_number)

df_players_finance = pd.DataFrame({
    "Player Name": player_names,
    "Market Value (m€)": player_values,
    "Fee (m€)": player_fees
})

df_players_finance

os.makedirs('data', exist_ok=True)
df_players_finance.to_csv('data/players_finance.csv', index=False)

End of pages 5


In [40]:
df_players_finance

,Player Name,Market Value (m€),Fee (m€)
0,Neymar,100.0,222.00
1,Philippe Coutinho,90.0,135.00
2,Ousmane Dembélé,33.0,135.00
3,Romelu Lukaku,50.0,84.70
4,Virgil van Dijk,30.0,84.65
...,...,...,...
95,Jhon Córdoba,7.0,17.00
96,Ismaïla Sarr,2.5,17.00
97,Ross Barkley,25.0,16.80
98,Medhi Benatia,15.0,16.70


## 1.2 Wyscout Player data

Please check that the wyscount data is downloaded and sits in /src/wyscout directory in this repository!

In [20]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
wyscout_dir = os.path.join(parent_dir, "src", "wyscout")

# Check if dir exists
assert os.path.exists(wyscout_dir), "Wyscout directory does not exist, download data!"

We define the function that caluclates the metrics with the given data files

In [96]:
import pandas as pd
import json
import os
import pathlib
import numpy as np
from scipy import stats

def decode_unicode_escapes(name):
    try:
        return name.encode('utf-8').decode('unicode_escape')
    except Exception as e:
        return name

def calculate_player_stats(players_file_path, Country):
    """
    Calculates performance metrics for football players from event data.

    Args:
    event_file_path (str): Path to the Event_<country>.json file.
    players_file_path (str): Path to the players.json file.

    Returns:
    pd.DataFrame: DataFrame containing player IDs, names, and calculated statistics.
    """
    
    event_file_path = os.path.join(wyscout_dir, "events" , f"events_{Country}.json")

    # Load event data
    with open(event_file_path) as file:
        events_data = json.load(file)
    events_df = pd.DataFrame(events_data)

    # Load player data
    with open(players_file_path) as file:
        players_data = json.load(file)
    players_df = pd.DataFrame(players_data)

    # TURNOVERS
    def turnovers(df):
        dribble_forward = df.loc[df["subEventName"] == "Ground attacking duel"]
        lost_dribble_forward = dribble_forward.loc[dribble_forward.apply(lambda x:{'id':701} in x.tags, axis = 1)]
        ldf = lost_dribble_forward.groupby(["playerId"]).eventId.count().reset_index()
        ldf.rename(columns = {'eventId':'lost_dribble_forward'}, inplace=True)
        
        passes = df.loc[df["eventName"] == "Pass"]
        lost_pass = passes.loc[passes.apply(lambda x:{'id':1802} in x.tags, axis = 1)]
        lp = lost_pass.groupby(["playerId"]).eventId.count().reset_index()
        lp.rename(columns = {'eventId':'lost_pass'}, inplace=True)

        turnovers_made = ldf.merge(lp, how = "outer", on = ["playerId"])
        return turnovers_made

    # CLEARANCES
    def clearance(df):
        clearance = df.loc[df["subEventName"] == "Clearance"]
        clearing_player = clearance.groupby(["playerId"]).eventId.count().reset_index()
        clearing_player.rename(columns = {'eventId':'clearances'}, inplace=True)
        return clearing_player

    # INTERCEPTIONS
    def interception(df):
        opp_pass = df.loc[df["eventName"] == "Pass"]
        interception = opp_pass.loc[opp_pass.apply(lambda x:{'id':1401} in x.tags, axis = 1)]
        intercepting_player = interception.groupby(["playerId"]).eventId.count().reset_index()
        intercepting_player.rename(columns = {'eventId':'interceptions'}, inplace=True)
        return intercepting_player

    # DUELS WON
    def defensiveDuelsWon(df):
        loose_duels = df.loc[df["subEventName"] == "Ground loose ball duel"]
        won_loose_duels = loose_duels.loc[loose_duels.apply(lambda x:{'id':703} in x.tags, axis = 1)]
        wld_player = won_loose_duels.groupby(["playerId"]).eventId.count().reset_index()
        wld_player.rename(columns = {'eventId':'loose_duels_won'}, inplace=True)
        
        def_ground_duels = df.loc[df["subEventName"].isin(["Ground defending duel"])]
        won_ground_duels = def_ground_duels.loc[def_ground_duels.apply(lambda x:{'id':703} in x.tags, axis = 1)]
        wgd_player = won_ground_duels.groupby(["playerId"]).eventId.count().reset_index()
        wgd_player.rename(columns = {'eventId':'def_ground_duels_won'}, inplace=True)

        air_duels = df.loc[df["subEventName"].isin(["Air duel"])]
        won_air_duels = air_duels.loc[air_duels.apply(lambda x:{'id':703} in x.tags, axis = 1)]
        wad_player = won_air_duels.groupby(["playerId"]).eventId.count().reset_index()
        wad_player.rename(columns = {'eventId':'air_duels_won'}, inplace=True)
        
        duels_won = wgd_player.merge(wld_player, how = "outer", on = ["playerId"]).merge(wad_player, how = "outer", on = ["playerId"])
        return duels_won

    # Apply functions
    turnover_stats = turnovers(events_df)
    clearance_stats = clearance(events_df)
    interception_stats = interception(events_df)
    duels_stats = defensiveDuelsWon(events_df)

    # Merge all statistics into a single DataFrame
    stats_df = pd.DataFrame(events_df["playerId"].unique(), columns=["playerId"])
    for stat_df in [turnover_stats, clearance_stats, interception_stats, duels_stats]:
        stats_df = stats_df.merge(stat_df, how="left", on=["playerId"])

    # Merge with player minutes played
    # path_minutes = os.path.join(str(pathlib.Path().resolve().parents[0]), 'DM-Football', 'src', 'wyscout', "minutes_played", 'minutes_played_per_game_England.json') 
    path_minutes = os.path.join(wyscout_dir, "minutes_played" , f"minutes_played_per_game_{Country}.json")
    
    with open(path_minutes) as f:
        minutes_per_game = json.load(f)
    minutes_df = pd.DataFrame(minutes_per_game)
    minutes = minutes_df.groupby(["playerId"]).minutesPlayed.sum().reset_index()

    stats_df = stats_df.merge(minutes, how="left", on=["playerId"])
    stats_df = stats_df.fillna(0)
    stats_df = stats_df.loc[stats_df["minutesPlayed"] > 400]

    # Calculate per 90 statistics
    for column in ['lost_dribble_forward', 'lost_pass', 'clearances', 'interceptions', 'loose_duels_won', 'def_ground_duels_won', 'air_duels_won']:
        stats_df[column + "_per90"] = stats_df.apply(lambda x: (x[column]*90)/x["minutesPlayed"] if x["minutesPlayed"] > 0 else 0, axis=1)

    # Merge with player names
    players_df['fullName'] = players_df['firstName'].fillna('') + ' ' + players_df['middleName'].fillna('') + ' ' + players_df['lastName'].fillna('')
    
    players_df['fullName'] = players_df['fullName'].str.strip().apply(decode_unicode_escapes)

    players_df = players_df[["wyId", "fullName"]]
    players_df.rename(columns={"wyId": "playerId"}, inplace=True)
    final_df = stats_df.merge(players_df, how="left", on="playerId")

    features = {
        "playerId": "Player_Id",
        "fullName": "Full_Name",
        "air_duels_won_per90": "Air_Duels_Won",
        "interceptions_per90": "Interceptions",
        "clearances_per90": "Clearances",
        "lost_pass_per90": "Turnovers",
        "loose_duels_won_per90": "Loose_Ball_Duels_Won",
        "def_ground_duels_won_per90": "Defensive_Duels_Won"
    }

    final_df = final_df[list(features.keys())]
    final_df.rename(columns=features, inplace=True)

    final_df.set_index("Player_Id", inplace=True)

    return final_df

Next step, we iterate over all countries and caclulcate player statstics. Outcome will be one big data file

In [97]:
player_stats_df = pd.DataFrame()

countries = ["England", "France", "Germany", "Italy", "Spain"]

players_file_path = os.path.join(wyscout_dir, "player.json")
events_dir = os.path.join(wyscout_dir, "events")


for country in countries:
        print("Processing", country)
        country_stats_df = calculate_player_stats(players_file_path, country)
        player_stats_df = pd.concat([player_stats_df, country_stats_df]).drop_duplicates()

Processing England
Processing France
Processing Germany
Processing Italy
Processing Spain


In [101]:
player_stats_df.to_csv("player_stats.csv")

In [88]:
player_stats_df[player_stats_df["fullName"].astype(str).str.contains("Holgate")]


,playerId,lost_dribble_forward,lost_pass,clearances,interceptions,def_ground_duels_won,loose_duels_won,air_duels_won,minutesPlayed,lost_dribble_forward_per90,lost_pass_per90,clearances_per90,interceptions_per90,loose_duels_won_per90,def_ground_duels_won_per90,air_duels_won_per90,fullName
270,355937,6.0,119.0,52.0,28.0,18.0,21.0,44.0,1239.0,0.435835,8.644068,3.77724,2.033898,1.525424,1.307506,3.196126,Mason Holgate
